# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Utilize statistical modeling and machine learning to identify trends and predict market movements for AAPL"}


> Entering new CrewAgentExecutor chain...
I should start by reading the latest market data for AAPL to analyze trends and predict movements.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/aapl"} 

marketwatch.comPlease enable JS and disable any ad blocker



Thought: I have gathered all the necessary information from the Data Analyst and Trading Strategy Developer. Now, I need to compile the potential trading strategies for AAPL that align with the user's risk tolerance and trading preferences.

Final Answer: 
Based on the insights provided by the Data Analyst on significant market opportunities and threats for AAPL, and considering the user's defined risk tolerance and trading preferences for Day Trading, the potential trading strategies for AAPL are as follows:
1. Momentum Trading Strategy: This strategy involves buying stocks that are showing upward momentum and selling them as soon as they show signs of reversal. This strategy aligns with the user's preference for Day Trading.
2. Breakout Trading Strategy: This strategy involves taking positions in a stock when it breaks out of a predefined range. This strategy can be effective for Day Trading and aligns with the user's risk tolerance.
3. Gap Trading Strategy: This strategy involves tr

Final Answer: 

Based on the analysis conducted by the Trading Strategy Developer, the following detailed execution plans are suggested for AAPL considering current market conditions and optimal pricing:

1. Momentum Trading Strategy:
- Buy AAPL stocks when they show upward momentum and sell as soon as signs of reversal are detected. Execute trades based on intraday price movements and volume indicators to capitalize on short-term price fluctuations.
- Recommended execution time: During active trading hours when volume and price movements are high.

2. Breakout Trading Strategy:
- Take positions in AAPL when it breaks out of a predefined range, indicating a potential price movement in the same direction. Utilize technical indicators such as moving averages and support/resistance levels to confirm breakouts.
- Recommended execution time: Monitor AAPL price movements during the opening hours to identify breakout opportunities.

3. Gap Trading Strategy:
- Trade AAPL stocks based on price 

Thought: I have gathered information on various trading strategies for AAPL from the search results. Now, I need to analyze the potential risks associated with each strategy and suggest mitigation strategies.

Action: Read website content
Action Input: {"website_url": "https://app.tradingsim.com/blog/apple-stock-3-simple-day-trading-strategies/"} 

Apple Stock: 3 Simple Day Trading Strategies – – Tradingsim
 Start Trial
Skip to content
Apple Stock: 3 Simple Day Trading Strategies
Apple Inc, trading on the NASDAQ exchange is one of the most popular stocks preferred by both intraday traders and long term investors. Investors flock to APPL stock for a number of reasons ranging from its high liquidity to its market capitalization and the strong fundamentals.
Apple Inc. set a new record of sorts as it created history with a market capitalization of one trillion U.S. dollars. The only other company to hit a one trillion market capitalization was PetroChina which briefly hit the milestone in 

Final Answer: 
After analyzing the information gathered from the search results on AAPL trading strategies, I have identified three simple day trading strategies for Apple stock: the two day high low strategy, trading ranges during low volatility, and the gap fill strategy. Each strategy presents its own risks and potential rewards. 

For the two day high low strategy, traders need to be prepared ahead of the market open and avoid major news events that could disrupt the strategy. It is essential to monitor the charts closely and be aware of potential price range movements. Mitigation strategies include setting stops at the previous session's low and aiming for a 1:2 risk/reward ratio.

Trading ranges during low volatility strategy requires traders to identify overbought and oversold levels using oscillators like RSI or Stochastics. While this strategy can provide steady profits, traders may face fake moves and more losing trades. Practicing on a demo account before trading live is rec

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

After analyzing the information gathered from the search results on AAPL trading strategies, I have identified three simple day trading strategies for Apple stock: the two day high low strategy, trading ranges during low volatility, and the gap fill strategy. Each strategy presents its own risks and potential rewards. 

For the two day high low strategy, traders need to be prepared ahead of the market open and avoid major news events that could disrupt the strategy. It is essential to monitor the charts closely and be aware of potential price range movements. Mitigation strategies include setting stops at the previous session's low and aiming for a 1:2 risk/reward ratio.

Trading ranges during low volatility strategy requires traders to identify overbought and oversold levels using oscillators like RSI or Stochastics. While this strategy can provide steady profits, traders may face fake moves and more losing trades. Practicing on a demo account before trading live is recommended.

The gap fill strategy involves trading the gaps that form between the previous day's close and the current day's open. Traders need to anticipate gaps forming before the market open, set stops strategically, and be prepared for price squeezes. Using a standard oscillator with intraday settings can help in picking better trades.

Overall, traders should choose a strategy that aligns with their personality and trading style, whether it's visually identifying gaps or trading volatility breakouts. Discipline and control are key to avoiding overtrading and maximizing profits with these simple day trading strategies for AAPL.